In [28]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Nov  1 12:27:43 2022

@author: schakrab
"""




import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from ipywidgets import interactive, interact, Layout, VBox, HBox
from IPython.display import display, clear_output




#base_dir = '/Users/schakrab/Desktop/Paper/input_folder/'
#City Initials
INIT     = 'HOU'
out_dir  = '/Users/schakrab/Desktop/Paper/input_folder/outputs/'+INIT+'/'
########################################################################################################################################################
#command lines### list of names of the cities and state ### Hardparse
########################################################################################################################################################
#Directories



# read masks
MU = np.loadtxt(out_dir+'CITY_05x06.csv', comments="#", delimiter=",", unpack=False)
SU = np.loadtxt(out_dir+'STATE_05x06.csv', comments="#", delimiter=",", unpack=False)
# The state mask is equivalent to the city + county + state. Areas that are double counted are set to one.
SU = MU  + SU
SU[SU > 1] = 1
# Create a mask of ones for the whole domain
DU = SU * 0 +1

names = ['Municipal', 'State','North America']
masks = [MU, SU, DU]


pm_cont = (Dataset(out_dir +INIT+'_PM_NO_O3.nc', mode='r').variables['pm_cont'][:])
pm_cont_lb = (Dataset(out_dir +INIT+'_PM_NO_O3.nc', mode='r').variables['pm_cont_lb'][:])
pm_cont_ub = (Dataset(out_dir +INIT+'_PM_NO_O3.nc', mode='r').variables['pm_cont_ub'][:])
o3_cont =(Dataset(out_dir +INIT+'_PM_NO_O3.nc', mode='r').variables['o3_cont'][:])
o3_cont_lb = (Dataset(out_dir +INIT+'_PM_NO_O3.nc', mode='r').variables['o3_cont_lb'][:])
o3_cont_ub =(Dataset(out_dir +INIT+'_PM_NO_O3.nc', mode='r').variables['o3_cont_ub'][:])
no_cont = (Dataset(out_dir +INIT+'_PM_NO_O3.nc', mode='r').variables['no_cont'][:])
no_cont_lb = (Dataset(out_dir +INIT+'_PM_NO_O3.nc', mode='r').variables['no_cont_lb'][:])
no_cont_ub = (Dataset(out_dir +INIT+'_PM_NO_O3.nc', mode='r').variables['no_cont_ub'][:])



# create slider widgets  
a=widgets.IntSlider(min=-100, max=100, step=5, value=0,description='Transport:',layout=widgets.Layout(width='100%')) 
clear_output()
b=widgets.IntSlider(min=-100, max=100, step=5, value=0,description='Energy:',layout=widgets.Layout(width='100%'))
c=widgets.IntSlider(min=-100, max=100, step=5, value=0,description='Agriculture:',layout=widgets.Layout(width='100%'))
d=widgets.IntSlider(min=-100, max=100, step=5, value=0,description='Industry:',layout=widgets.Layout(width='100%'))
# create species selector 
species = widgets.Select(options=['PM25','O3','NO2'],description='Species:',disabled=False)
# specify handle colors of sliders
a.style.handle_color = 'lightgreen' 
b.style.handle_color = 'lightblue' 
c.style.handle_color = 'red' 
d.style.handle_color = 'black' 



def f(a, b, c, d, species): 
    # create a list of scale values from widgets
    scales = [a,b,c,d]
    # species specific information
    if species == 'PM25':
        J = pm_cont
        J_lb = pm_cont_lb
        J_ub = pm_cont_ub        
        ymax = 3000
        lab = ' Deaths'
        longlab = 'Premature Deaths'
        levels = np.linspace(-30,30,21)
    if species == 'O3':
        J = o3_cont
        J_lb = o3_cont_lb
        J_ub = o3_cont_ub          
        ymax = 1000
        lab = ' Deaths'        
        longlab = 'Premature Deaths'        
        levels = np.linspace(-10,10,21)
    if species == 'NO2':
        J = no_cont
        J_lb = no_cont_lb
        J_ub = no_cont_ub          
        ymax = 35000
        lab = ' Cases'        
        longlab = 'New Pediatric Asthma Cases'
        levels = np.linspace(-150,150,21)
    # intialize and calculate contribution for pollutant health impacts
    dJ = J[0,:,:,:].sum(axis=0)*0                
    dJ_lb = J_lb[0,:,:,:].sum(axis=0)*0 
    dJ_ub = J_ub[0,:,:,:].sum(axis=0)*0     
    for i, scale in enumerate(scales):
        dJ = dJ + J[i,:,:,:].sum(axis=0)*scale
        dJ_lb = dJ_lb + J_lb[i,:,:,:].sum(axis=0)*scale
        dJ_ub = dJ_ub + J_ub[i,:,:,:].sum(axis=0)*scale  
    #print(np.shape(dJ),np.shape(masks))    
    # intialize and calculate contributions from specific regions
    values = [0,0,0]
    values_lb = [0,0,0]
    values_ub = [0,0,0]    
    for i,mask in enumerate(masks):
        values[i] = np.multiply(dJ,mask).sum()/100
        values_lb[i] = np.multiply(dJ_lb,mask).sum()/100
        values_ub[i] = np.multiply(dJ_ub,mask).sum()/100        
    # create bar plot
#    plt.close()
#    fig, ax = plt.subplots(figsize=(10,20))  
    fig = plt.figure(figsize=(10,20))
    ax = fig.add_subplot(2, 1, 1)
    bar1 = plt.bar(names,values,color=['cyan', 'red', 'green', 'blue', 'cyan'])    
    error = [[0 for x in range(3)] for y in range(2)]
    error[0][0]=values[0]-values_lb[0]
    error[1][0]=values_ub[0]-values[0]
    error[0][1]=values[1]-values_lb[1]
    error[1][1]=values_ub[1]-values[1]       
    error[0][2]=values[2]-values_lb[2]
    error[1][2]=values_ub[2]-values[2]       
    plt.errorbar(names,values,error,ls='none',capsize=50,ecolor=[0,0,0])
    plt.title('Pollution Health Impacts in City')
    plt.ylabel('Change in Annual '+longlab)
    plt.ylim(-ymax, ymax)    
    ax.axhline(0, color='grey', linewidth=0.8)    
    # Add counts above the two bar graphs
    for c,rect in enumerate(bar1):
        height = rect.get_height()
        if height < 0:
            plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height:.0f}'+lab, ha='center', va='top')            
        else:
            plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height:.0f}'+lab, ha='center', va='bottom')        
    

    ax = fig.add_subplot(2, 1, 2)
    plt.contourf(dJ/100,levels)
    plt.set_cmap('viridis')
    plt.colorbar()
    plt.show()
    
    
    
# create interactive plot    
#plt.close('all')
interactive_plot = interactive(f,a=a,b=b,c=c,d=d,species=species)
clear_output()
# specify size of plot by taking the output
output = interactive_plot.children[-1]
output.layout.height = '1200px'
output.layout.width = '800px'
# display output in "HBox"
HBox([interactive_plot])    
    
    #city?how to resd this scenario? talk to OMar precond, assumption
    #citname as a function call